In [178]:
import pandas as pd
import numpy as np
import folium
from folium import plugins
from folium.plugins import HeatMap
import branca
from collections import defaultdict

In [179]:
earthquake_data = pd.read_csv("../data/Cali_earthquakes.csv")
earthquake_data= earthquake_data.dropna()

In [180]:
earthquake_data.head()

,X,Y,OBJECTID_1,year,month,day,hour,minute,second,lat,lon,depth,magnitude,OBJECTID
0,-1.313570e+07,4.028803e+06,1,1769,7,28,99,99,99.99,34.00,-118.00,99.0,6.0,1
1,-1.358098e+07,4.467022e+06,2,1781,10,9,99,99,99.99,37.20,-122.00,99.0,5.5,2
2,-1.353311e+07,4.421003e+06,3,1800,10,11,99,99,99.99,36.87,-121.57,99.0,5.5,3
3,-1.311344e+07,3.882039e+06,4,1800,11,22,21,30,9900.00,32.90,-117.80,99.0,6.3,4
4,-1.314683e+07,4.055690e+06,5,1803,4,99,99,99,99.99,34.20,-118.10,99.0,5.5,5


In [181]:
magnitude_counts = earthquake_data['magnitude'].value_counts()

print(magnitude_counts)

magnitude
5.00    83
5.50    80
5.10    43
6.00    38
5.80    37
        ..
5.79     1
6.08     1
5.39     1
6.22     1
5.36     1
Name: count, Length: 127, dtype: int64


In [182]:
# get the center and make the map object
m = folium.Map(location=(36.7783, -119.4179), zoom_start=6)

# go through each quake in set, make circle, and add to map.
for i in range(len(earthquake_data)):
    if earthquake_data.iloc[i]['magnitude'] > 3:
        folium.Circle(
            location=[earthquake_data.iloc[i]['lat'], earthquake_data.iloc[i]['lon']],
            radius=earthquake_data.iloc[i]['magnitude'] * 700,
            weight=1,  # thickness of the border
            color='red',  # this is the color of the border
            opacity=0.3,  # this is the alpha for the border
            fill_color='red',  # fill is inside the circle
            fill_opacity=0.1,  # we will make that less opaque so we can see layers

        ).add_to(m)
m

In [183]:
heatmap_data= []


steps = 5
vmin = 5
vmax = 7.9

#colormap = branca.colormap.linear.YlOrRd_09.scale(0,1).to_step(steps)
colors = ['#ffffb2', '#fecc5c', '#fd8d3c', '#f03b20', '#bd0026']
colormap = branca.colormap.LinearColormap(colors, vmin=vmin, vmax=vmax)

# Scale the colormap to the specified number of steps
colormap = colormap.to_step(steps)

gradient_map=defaultdict(dict)
keys = np.linspace(0,1,5)
print(keys)
for i,key in enumerate(keys):
    gradient_map[key] = colors[i]

# for i in range(steps):
#     print(1/steps*i)
#     gradient_map[1/steps*i] = colors[i]
#     #map.rgb_hex_str((1/steps*i)*255)

#gradient_map = colormap.to_linear()
print(gradient_map)

for i in range(len(earthquake_data)):
    if earthquake_data.iloc[i]['magnitude'] > 3:
        # Append the coordinates and magnitude to the heatmap_data list
        heatmap_data.append([earthquake_data.iloc[i]['lat'], earthquake_data.iloc[i]['lon'], earthquake_data.iloc[i]['magnitude']])
        

# Create a folium Map object
m = folium.Map(location=(36.7783, -119.4179), zoom_start=6)

# Create the HeatMap layer with the heatmap_data
plugins.HeatMap(heatmap_data,
                 gradient=gradient_map
                 ).add_to(m)

colormap.add_to(m)
m


[0.   0.25 0.5  0.75 1.  ]
defaultdict(<class 'dict'>, {0.0: '#ffffb2', 0.25: '#fecc5c', 0.5: '#fd8d3c', 0.75: '#f03b20', 1.0: '#bd0026'})


In [184]:
import branca
import folium
from folium import plugins
import numpy as np
from collections import defaultdict

# Assuming earthquake_data is already loaded

# Step 1: Determine vmin and vmax
vmin = earthquake_data[earthquake_data['magnitude'] > 5]['magnitude'].min()
vmax = earthquake_data[earthquake_data['magnitude'] > 5]['magnitude'].max()

# Step 2: Set up the colormap
colors = ['#FFFFB2', '#FECC5C', '#FD8D3C', '#F03B20', '#BD0026']
colormap = branca.colormap.LinearColormap(colors, vmin=vmin, vmax=vmax).to_step(steps)

# Step 3: Create the gradient map
gradient_map = {}
for i, color in enumerate(colors):
    gradient_map[i / (steps - 1)] = color

# Step 4: Filter earthquake data and display the heatmap
heatmap_data = [
    [row['lat'], row['lon'], row['magnitude']]
    for _, row in earthquake_data.iterrows() 
    if row['magnitude'] > 5
]

m = folium.Map(location=(36.7783, -119.4179), zoom_start=6)
plugins.HeatMap(heatmap_data, gradient=gradient_map).add_to(m)
colormap.add_to(m)
m